In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType
import psycopg2
from pyspark.sql import DataFrame

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define schema for the flat file
flat_file_schema = StructType([
    StructField("AverageRate", FloatType(), True),
    StructField("CurrencyID", StringType(), True),
    StructField("CurrencyDate", DateType(), True),
    StructField("EndOfDayRate", FloatType(), True)
])

# Load data from the flat file source
try:
    logger.info("Loading data from flat file source.")
    flat_file_df = spark.read.format("csv") \
        .option("header", "true") \
        .schema(flat_file_schema) \
        .load("/Volumes/genai_demo/ssis/ssis/SampleCurrencyData.txt")
    logger.info("Flat file data loaded successfully.")
except Exception as e:
    logger.error(f"Error loading flat file data: {e}")
    raise

# Load DimCurrency1 table from Unity Catalog
try:
    logger.info("Loading DimCurrency1 table from Unity Catalog.")
    dim_currency_df = spark.table("sqlserver_catalog.dbo.dimcurrency1")
    logger.info("DimCurrency1 table loaded successfully.")
except Exception as e:
    logger.error(f"Error loading DimCurrency1 table: {e}")
    raise

# Load DimDate1 table from Unity Catalog
try:
    logger.info("Loading DimDate1 table from Unity Catalog.")
    dim_date_df = spark.table("sqlserver_catalog.dbo.dimdate1")
    logger.info("DimDate1 table loaded successfully.")
except Exception as e:
    logger.error(f"Error loading DimDate1 table: {e}")
    raise

# Perform lookup transformation for Currency Key
try:
    logger.info("Performing lookup transformation for Currency Key.")
    enriched_currency_df = flat_file_df.join(
        dim_currency_df,
        flat_file_df.CurrencyID == dim_currency_df.CurrencyAlternateKey,
        "inner"
    ).select(
        flat_file_df["*"],
        dim_currency_df["CurrencyAlternateKey"]
    )
    logger.info("Currency Key lookup transformation completed successfully.")
except Exception as e:
    logger.error(f"Error in Currency Key lookup transformation: {e}")
    raise

# Perform lookup transformation for Date Key
try:
    logger.info("Performing lookup transformation for Date Key.")
    enriched_date_df = enriched_currency_df.join(
        dim_date_df,
        enriched_currency_df.CurrencyDate == dim_date_df.FullDateAlternateKey,
        "inner"
    ).select(
        enriched_currency_df["*"],
        dim_date_df["EnglishDayNameOfWeek"],
        dim_date_df["EnglishMonthName"],
        dim_date_df["DateKey"],
        dim_date_df["FullDateAlternateKey"]
    )
    logger.info("Date Key lookup transformation completed successfully.")
except Exception as e:
    logger.error(f"Error in Date Key lookup transformation: {e}")
    raise

# Drop the existing target table if it exists
try:
    logger.info("Dropping existing target table if it exists.")
    spark.sql("DROP TABLE IF EXISTS sqlserver_catalog.dbo.result")
    logger.info("Existing target table dropped successfully.")
except Exception as e:
    logger.error(f"Error dropping existing target table: {e}")
    raise

# Write the transformed data to the target Unity Catalog table
try:
    logger.info("Writing transformed data to the target Unity Catalog table.")
    enriched_date_df.write.format("delta").mode("overwrite").saveAsTable("sqlserver_catalog.dbo.result")
    logger.info("Transformed data written to target table successfully.")
except Exception as e:
    logger.error(f"Error writing transformed data to target table: {e}")
    raise

logger.info("ETL process completed successfully.")
